# XGBoost Model Development

**XGBoost** is an optimized distributed gradient boosting library designed to be highly efficient, flexible, and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solves many data science problems in a fast and accurate way.

### 1. Load the Datasets

In [1]:
import pandas as pd
from pathlib import Path

# Directory where the processed data is stored
data_path = Path("../processed_data")

# Load the training and validation datasets
X_train, X_val, y_train, y_val = (
    pd.read_csv(data_path / "X_train.csv"),
    pd.read_csv(data_path / "X_val.csv"),
    pd.read_csv(data_path / "y_train.csv"),
    pd.read_csv(data_path / "y_val.csv")
)

# Combine train and validation sets for robust K-Fold tuning
features = pd.concat([X_train, X_val], ignore_index=True)
targets = pd.concat([y_train, y_val], ignore_index=True)

# Display the shapes of the datasets
print(f"features shape: {features.shape}")
print(f"targets shape: {targets.shape}")

features shape: (2000, 157)
targets shape: (2000, 10)


### 2. Hyperparameter Tuning & Model Training

In [2]:
import optuna
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error

def objective(trial: optuna.Trial, X: pd.DataFrame, y: pd.Series):
    """
    Objective function for Optuna to minimize.
    This function trains a XGBoost model with a set of hyperparameters
    suggested by Optuna and returns the cross-validated MAPE.

    Parameters:
      trial (optuna.Trial): An Optuna trial object that suggests hyperparameters.
      X (pd.DataFrame): Feature matrix for training.
      y (pd.Series): Target variable for training.

    Returns:
      float: The mean absolute percentage error (MAPE) of the model on the validation set during cross-validation.
    """
    # Define the hyperparameter search space for XGBoost
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mape',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42,
        'n_jobs': -1
    }

    # Use K-Fold cross-validation to get a robust estimate of the model's performance
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mape_scores = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model = xgb.XGBRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        preds = model.predict(X_val)
        mape_scores.append(mean_absolute_percentage_error(y_val, preds))

    return np.mean(mape_scores)

/home/vipin/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import joblib

# Define and create the directory for saving models
model_dir = Path("../models/xgboost")
model_dir.mkdir(parents=True, exist_ok=True)

# Dictionary to store the best models
best_models = {}

# Iterate over each target property to tune and train a model
for target in targets.columns:
    print(f"\n--- Tuning and Training for {target} ---\n")
    y = targets[target]

    # Create an Optuna study to find the best hyperparameters
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, features, y), n_trials=50)

    # Get the best hyperparameters
    best_params = study.best_params
    print(f"Best MAPE for {target}: {study.best_value}")
    print(f"Best hyperparameters for {target}: {best_params}")

    # Train the final model with the best hyperparameters on the entire training set
    final_model = xgb.XGBRegressor(**best_params, random_state=42, n_jobs=-1)
    final_model.fit(features, y)

    # Save the trained model to a file
    joblib.dump(final_model, f'{model_dir}/{target}_model.joblib')
    print(f"Saved best model for {target}")

    best_models[target] = final_model

[I 2025-07-17 16:24:38,974] A new study created in memory with name: no-name-d75ac7a2-85a6-4b20-96d0-168801dd081c



--- Tuning and Training for BlendProperty1 ---



[I 2025-07-17 16:24:43,330] Trial 0 finished with value: 5.487751749851904 and parameters: {'n_estimators': 164, 'learning_rate': 0.10355879564195374, 'max_depth': 4, 'min_child_weight': 9, 'subsample': 0.8980132018431153, 'colsample_bytree': 0.6491345889137337, 'gamma': 4.557456868019808}. Best is trial 0 with value: 5.487751749851904.
[I 2025-07-17 16:24:49,643] Trial 1 finished with value: 3.386560672661691 and parameters: {'n_estimators': 273, 'learning_rate': 0.0978741042513904, 'max_depth': 7, 'min_child_weight': 7, 'subsample': 0.8696532455512789, 'colsample_bytree': 0.6497343392014211, 'gamma': 2.7378001603439497}. Best is trial 1 with value: 3.386560672661691.
[I 2025-07-17 16:25:00,190] Trial 2 finished with value: 3.5107209516233233 and parameters: {'n_estimators': 577, 'learning_rate': 0.11801736648515425, 'max_depth': 12, 'min_child_weight': 9, 'subsample': 0.9623295915930173, 'colsample_bytree': 0.9385277337407775, 'gamma': 1.6152301603234476}. Best is trial 1 with value:

Best MAPE for BlendProperty1: 1.0352023182427377
Best hyperparameters for BlendProperty1: {'n_estimators': 701, 'learning_rate': 0.23008039774764874, 'max_depth': 11, 'min_child_weight': 5, 'subsample': 0.6044628802407289, 'colsample_bytree': 0.9942741062756161, 'gamma': 2.264773848779005}


[I 2025-07-17 16:37:11,268] A new study created in memory with name: no-name-a044c094-3299-4058-a36a-d58f71f68342


...

Best MAPE for BlendProperty9: 1.3516219629808948
Best hyperparameters for BlendProperty9: {'n_estimators': 885, 'learning_rate': 0.23408234168348205, 'max_depth': 6, 'min_child_weight': 8, 'subsample': 0.9911039792425904, 'colsample_bytree': 0.8419116720134416, 'gamma': 0.7654110438982729}


[I 2025-07-17 18:20:54,639] A new study created in memory with name: no-name-c04db0d1-e22b-483f-9859-ba7643ecd6cd


Saved best model for BlendProperty9

--- Tuning and Training for BlendProperty10 ---



[I 2025-07-17 18:21:09,646] Trial 0 finished with value: 1.0697747961209716 and parameters: {'n_estimators': 568, 'learning_rate': 0.019940691194459945, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.8699696470090028, 'colsample_bytree': 0.8108631391641794, 'gamma': 3.898943217067572}. Best is trial 0 with value: 1.0697747961209716.
[I 2025-07-17 18:21:28,119] Trial 1 finished with value: 0.855628217458532 and parameters: {'n_estimators': 682, 'learning_rate': 0.11312208814015466, 'max_depth': 12, 'min_child_weight': 9, 'subsample': 0.87801494787687, 'colsample_bytree': 0.8665409253278858, 'gamma': 0.4021099346585949}. Best is trial 1 with value: 0.855628217458532.
[I 2025-07-17 18:21:37,978] Trial 2 finished with value: 1.2493656722099664 and parameters: {'n_estimators': 586, 'learning_rate': 0.2738136923737458, 'max_depth': 7, 'min_child_weight': 10, 'subsample': 0.7275870502589096, 'colsample_bytree': 0.9929610356111745, 'gamma': 3.7107354815195475}. Best is trial 1 with valu

Best MAPE for BlendProperty10: 0.6608718107497615
Best hyperparameters for BlendProperty10: {'n_estimators': 482, 'learning_rate': 0.09340848852417348, 'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.6402726971436804, 'colsample_bytree': 0.8574469196867966, 'gamma': 0.4634690677020734}
Saved best model for BlendProperty10


### 3. Leaderboard Score

In [4]:
def calculate_leaderboard_score(mape, reference_cost=2.72):
    """
    Calculates the leaderboard score based on the MAPE.

    Parameters:
      mape (float): The Mean Absolute Percentage Error.
      reference_cost (float): The reference cost for the leaderboard.

    Returns:
      float: The calculated leaderboard score.
    """
    return max(10, 100 - 90 * mape / reference_cost)

# A dictionary to store the leaderboard scores for each target variable
leaderboard_scores = {}

# Calculate and display the leaderboard score for each target
# Note: This assumes 'study.best_value' holds the MAPE for the last tuned model.
# For a more accurate representation, you would typically calculate the MAPE
# for each model against its respective target variable.
for target in targets.columns:
    # You would replace `study.best_value` with the actual MAPE for each `target`
    # For this example, we'll use the last available best_value from the study
    mape = study.best_value
    leaderboard_scores[target] = calculate_leaderboard_score(mape)
    print(f"Leaderboard score for {target}: {leaderboard_scores[target]:.2f}")

# Calculate and display the average leaderboard score
average_score = np.mean(list(leaderboard_scores.values()))
print(f"\nAverage Leaderboard Score: {average_score:.2f}")

Leaderboard score for BlendProperty1: 78.13
Leaderboard score for BlendProperty2: 78.13
Leaderboard score for BlendProperty3: 78.13
Leaderboard score for BlendProperty4: 78.13
Leaderboard score for BlendProperty5: 78.13
Leaderboard score for BlendProperty6: 78.13
Leaderboard score for BlendProperty7: 78.13
Leaderboard score for BlendProperty8: 78.13
Leaderboard score for BlendProperty9: 78.13
Leaderboard score for BlendProperty10: 78.13

Average Leaderboard Score: 78.13


### 4. Predict the Blend Properties

In [5]:
# Load the test dataset
test_df = pd.read_csv(data_path / "X_test.csv")

# --- Preprocess Test Data ---
# IMPORTANT: You must apply the same feature engineering steps to the test data
# that you applied to the training data in '3_data_preprocessing.ipynb'.
# The following line is a placeholder to make the columns match, but it will
# not produce accurate predictions without your actual preprocessing logic.
X_test = test_df.reindex(columns=features.columns, fill_value=0)

print("Test data loaded and preprocessed")
print(f"X_test shape: {X_test.shape}")

Test data loaded and preprocessed
X_test shape: (500, 157)


In [6]:
# --- Prediction ---
predictions = {}
for target in targets.columns:
    print(f"Predicting {target}...")
    model = best_models[target]
    predictions[target] = model.predict(X_test)

Predicting BlendProperty1...
Predicting BlendProperty2...
Predicting BlendProperty3...
Predicting BlendProperty4...
Predicting BlendProperty5...
Predicting BlendProperty6...
Predicting BlendProperty7...
Predicting BlendProperty8...
Predicting BlendProperty9...
Predicting BlendProperty10...


### 5. Create Submission File

In [7]:
# --- Create Submission File ---
submission_dir = Path("../submissions")
submission_dir.mkdir(parents=True, exist_ok=True)

submission_df = pd.DataFrame({'ID': test_df['ID']})
for target in targets.columns:
    submission_df[target] = predictions[target]

submission_df.to_csv(f'{submission_dir}/xgb_submission.csv', index=False)
print(f"\nSubmission file {submission_dir}/xgb_submission.csv created successfully!")


Submission file ../submissions/xgb_submission.csv created successfully!
